# Neste notebook faremos a simulação por dinâmica molecular do gás argônio superfluido à 94 K e 1 atm.

Antes de dar início à simulação, deveremos instalar o pacote M3L responsável pela dinâmica molecular. Descomente a linha abaixo caso ele ainda não foi instalado no seu ambiente virtual.

In [1]:
#pip install -i https://test.pypi.org/simple/ m3l --extra-index-url https://pypi.org/simple/

In [2]:
# consumo inicial de memoria
import psutil
mem = psutil.virtual_memory().used

In [3]:
!pwd

/home/flaviano/Documentos/GitHub/Notebooks/M3L/argon


## Etapa 1
O primeiro passo consiste em definirmos o sistema físico contendo as coordenadas atômicas. Descomente as cinco primeiras linhas abaixo para fazer o download do arquivo "Ar_94K_1atm.json".

In [4]:
#url = 'https://raw.githubusercontent.com/flavianowilliams/M3L/development/src/notebooks/argon/Ar_94K_1atm.json'
#from urllib.request import urlopen
#
#file = urlopen(url)
#
#with open('Ar_94K_1atm.json','wb') as output:
#  output.write(file.read())
#
# carregando sistema físico
from m3l.structure import System as sys
import json
#
system = sys()
system.loadSystem('Ar_94K_1atm.json')
#
# convertendo as unidades de medida
system.convertUnits()

## Etapa 2
O segundo passo consiste em definirmos o modelo para o campo de força. Neste caso consideraremos uma molécula contendo apenas um sítio atômico com interações intermoleculares do tipo Van der Waals representado pela função Lennard-Jones de dois parâmetros, como mostra a figura abaixo.
![lennard-jones](../contrib/lennard-jones.jpeg)

In [5]:
# definindo o modelo de interação entre os átomos (campo de força)      
import m3l.force_field as ff
from m3l.force_field import Intermolecular as interff
from m3l.force_field import Molecules as mo
class Forces(ff.ForceField2):
    def __init__(self):
        super().__init__()
        self.van_der_waals = interff.sites(
            interff.site(1, 1, 0.2385, 3.4), #site ID, atomic charge, Lennard-Jones parameters
        )
        self.convertUnits()
#        self.molecules = mo.molecule(1),
#        )
#    def intermolecular(self.van_der_waals, rvdw = 11.0, rcoul = 11.0)
#        return self.van_der_waals
#        
model = Forces()
model()

(1, 0.0, 0.0, [[1, 1, 0.0003802000129863748, 6.42506882808646]])

## Etapa 3
O terceiro passo consiste em definir o modelo estatístico (ensemble canônico) que descreverá o sistema no equilíbrio termodinâmico.

In [6]:
# definindo o modelo estatístico (ensemble)
from m3l.molecular_dynamics import Ensemble as en
#
force_field = model()
#ensemble = en(temp_bath = 94.0, timestep = 1.0e-3, force_field = force_field, tstat = 2.0)
ensemble = en(temp_bath = 94.0, press_bath = 1.0, timestep = 1.0e-6, force_field = force_field, pstat = 2.0, tstat = 2.0, bfactor = 5e-5)
system = ensemble(system)
ensemble.force_field

(1, 0.0, 0.0, [[1, 1, 0.0003802000129863748, 6.42506882808646]])

## Etapa 4
O quarto passo consiste na dinâmica molecular, onde as coordenadas atômicas são recalculadas a cada passo da simulação em intervalos de tempo denominado de timestep.

In [7]:
# Etapa 4
# executando ciclo MD
import time
#
# executando looping
n_steps = 1000
i_step = 0
start = time.time()
for step in range(n_steps):
    system = ensemble(system)
#    if step%10 == 0:
#        print(system.virial, system.pressure, system.ekinetic)
end = time.time()
print(f'Elapsed time: {round(end - start, 0)} seconds')

Elapsed time: 2.0 seconds


## Etapa 5
Esta etapa é opcional, onde o autor poderá salvar em um arquivo JSON o sistema físico obtido no final da simulação. Como opção, o autor poderá converter as unidades de medida em valores mais convenientes para a leitura e melhor interpretação do ponto de vista didático.

In [8]:
# Etapa opcional
# convertendo para as unidades de medida de entrada
system.convertUnitsInv()
#
# salvando sistema em arquivo JSON
#system.save('Ar_94K_1atm.json')

In [9]:
# Etapa opcional
# visualizando o consumo de memória RAM
mem = psutil.virtual_memory().used - mem
print(f'{mem/1.e6:.0f} MB')

1 MB


Algumas propriedades termodinâmicas do sistema físico poderão ser lidas através dos atributos do objeto system.

In [10]:
# Etapa opcional
# Obtendo variáveis termodinâmicas do sistema
header = ['Propriedade', 'Valor', 'Unidade']
print(f"{header[0]:<25} {header[1]:<25} {header[2]:<25}")
print("-"*75)
propriedade = ['Energia potencial', 'Energia cinética', 'Temperatura', 'Pressão', 'Volume']
unidade = [r'kcal/mol', 'kcal/mol', 'Kelvin', 'atmosfera', 'Angstrom³']
valor = [system.epotential, system.ekinetic, system.temperature, system.pressure, system.volume]
#
for i in range(len(propriedade)):
    print(f"{propriedade[i]:<25} {valor[i]:<25} {unidade[i]:<25}")

Propriedade               Valor                     Unidade                  
---------------------------------------------------------------------------
Energia potencial         -388.0064834492432        kcal/mol                 
Energia cinética          71.96874439156826         kcal/mol                 
Temperatura               96.99622707487468         Kelvin                   
Pressão                   -65.89146535820771        atmosfera                
Volume                    11346.77333224905         Angstrom³                


In [11]:
# limpando memória RAM
del model
del ensemble
del system